# Cdiscount’s Image Classification Challenge PyTorch CNN starter (Work in progress)

- Based on code adapted from here:
https://www.kaggle.com/alekseit/pytorch-bson-dataset (All credist go to the respective writer https://www.kaggle.com/alekseit) 

- Hopefully will write a full CNN in the next few days. 


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

% reset -f
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

import torch
import sys
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split

print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)

import numpy
import numpy as np

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

import pandas
import pandas as pd

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)
%matplotlib inline

# !pip install psutil
import psutil
import os
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()

In [2]:
import os
import pandas as pd
import numpy as np
import bson
import cv2
from tqdm import tqdm
import struct
from PIL import Image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


def flaotTensorToImage(img, mean=0, std=1):
    """convert a tensor to an image"""
    img = np.transpose(img.numpy(), (1, 2, 0))
    img = (img*std+ mean)*255
    img = img.astype(np.uint8)    
    return img    

def toTensor(img):
    """convert a numpy array of shape HWC to CHW tensor"""
    img = img.transpose((2, 0, 1)).astype(np.float32)
    tensor = torch.from_numpy(img).float()
    return tensor/255.0    

In [3]:
def read_bson(bson_path, num_records, with_categories):
    """
    Reads BSON
    """
    rows = {}
    with open(bson_path, "rb") as f, tqdm(total=num_records) as pbar:
        offset = 0
        records_read = 0 
        while True:
            item_length_bytes = f.read(4)
            if len(item_length_bytes) == 0:
                break
            
            length = struct.unpack("<i", item_length_bytes)[0]
            f.seek(offset)
            item_data = f.read(length)
            assert len(item_data) == length

            item = bson.BSON.decode(item_data)
            product_id = item["_id"]
            num_imgs = len(item["imgs"])

            row = [num_imgs, offset, length]
            if with_categories:
                row += [item["category_id"]]
            rows[product_id] = row

            offset += length
            f.seek(offset)
            records_read += 1
            pbar.update()
    pbar.close()
    columns = ["num_imgs", "offset", "length"]
    if with_categories:
        columns += ["category_id"]

    df = pd.DataFrame.from_dict(rows, orient="index")
    df.index.name = "product_id"
    df.columns = columns
    df.sort_index(inplace=True)
    return df


def make_category_tables(categories_path):
    """
    Converts category name into an index [0, N-1]
    """
    categories_df = pd.read_csv(categories_path, index_col="category_id")
    categories_df["category_idx"] = pd.Series(range(len(categories_df)), index=categories_df.index)
    
    print ('categories_df:' + str(categories_df.head()))
    
    cat2idx = {}
    idx2cat = {}
    for ir in categories_df.itertuples():
        category_id = ir[0]
        category_idx = ir[4]
        cat2idx[category_id] = category_idx
        idx2cat[category_idx] = category_id
    print ('cat2idx:' + str(len(cat2idx)))
    print ('idx2cat:' + str(len(idx2cat)))
    return cat2idx, idx2cat

def get_obs(fname, offset, length):
    fobj = open(fname, 'rb')
    fobj.seek(offset)
    res = bson.BSON.decode(fobj.read(length))
    fobj.close()
    return res

class CdiscountDataset(Dataset):
    def __init__(self, dataset, split, transform):
        self.dataset = dataset
        self.metadata = split
        self.transform = transform

    def __getitem__(self, index):
        entry = self.metadata.iloc[index]
        num_imgs, offset, length, target = entry
        obs = get_obs(self.dataset, offset, length)
        keep = np.random.choice(len(obs['imgs']))
        byte_str = obs['imgs'][keep]['picture']
        img = cv2.imdecode(np.fromstring(byte_str, dtype=np.uint8), cv2.IMREAD_COLOR)

        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        img = self.transform(img)

        return img, target

    def __len__(self):
        return self.metadata.index.values.shape[0]

batch_size = 16 # on GTX 1080
global_epoches = 10
LR = 0.0005
MOMENTUM = 0.95
validationRatio=0.11    


In [4]:
# if __name__ == "__main__":
TRAIN_BSON_FILE = '../input/train.bson'
CATEGS = '../input/category_names.csv'
# Chenge this weh running on your machine
N_TRAIN = 7069896
BS = 32
N_THREADS = 0

# mapping the catigores into 0-5269 range
cat2idx, idx2cat = make_category_tables(CATEGS)
# print (cat2idx)
# print (idx2cat)


In [5]:
#Scanning the metadata
meta_data = read_bson(TRAIN_BSON_FILE, N_TRAIN, with_categories=True)
meta_data.category_id = np.array([cat2idx[ind] for ind in meta_data.category_id])
# meta_data = meta_data.iloc[np.arange(500)] # Remove this!!!
# Dataset and loader
transformations = transforms.Compose([transforms.ToTensor()])

train_ds = CdiscountDataset(TRAIN_BSON_FILE, meta_data,transformations)
train_loader = DataLoader(train_ds, batch_size=BS,num_workers=N_THREADS, shuffle=True)

print(train_loader)

# Let's go fetch some data!
# pbar = tqdm(total=len(train_loader))
# for i, (batch, target) in enumerate(loader):
#     pass
#     pbar.update()
# pbar.close()



In [ ]:
# https://github.com/Cognexa/cdiscount-kernel/blob/master/cdc/cdc_dataset.py
def split(self):
        """
        Split train data to train and validation sets and compute (category_id -> integer class) mapping.
        Run with `cxflow dataset split cdc`
        :return:
        """
        # read example headers
        logging.info('Reading examples metadata, this may take a minute or two')
        ids = []
        categories = []
        for example in bson.decode_file_iter(open(path.join(self._data_root, self.TRAIN_FILE), 'rb')):
            ids.append(example['_id'])
            categories.append(example['category_id'])

        # generate random split
        size = len(ids)
        train_size = int(size*(1-self._valid_percent))
        valid_size = (size-train_size)
        split = ['train']*train_size + ['valid']*valid_size
        random.shuffle(split)

        # save split
        split_df = pd.DataFrame({'id': ids, 'split': split})
        split_path = path.join(self._data_root, self._split_file)
        split_df.to_csv(split_path, index=False)
        logging.info('Split train-valid of size %s-%s was written to `%s`', train_size, valid_size, split_path)

        # save (category_id -> integer class) mapping
        categories = sorted(list(set(categories)))
        categories_df = pd.DataFrame({'category_id': categories, 'class': list(range(len(categories)))})
        categories_path = path.join(self._data_root, self.CATEGORIES_FILE)
        categories_df.to_csv(categories_path, index=False)
        logging.info('Categories mapping saved to `{}`'.format(categories_path))

In [6]:
import matplotlib.pyplot as plt

imagesToShow=4

for i, data in enumerate(train_loader, 0):
    lgr.info('i=%d: '%(i))            
    images, labels = data            
    num = len(images)

    ax = plt.subplot(1, imagesToShow, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')

    for n in range(num):
        image=images[n]
        label=labels[n]
#         print ('Label(s):' + str(label))
        plt.imshow (flaotTensorToImage(image))

    if i==imagesToShow-1:
        break    


In [7]:
class FullTrainningDataset(torch.utils.data.Dataset):
    def __init__(self, full_ds, offset, length):
        self.full_ds = full_ds
        self.offset = offset
        self.length = length
        assert len(full_ds)>=offset+length, Exception("Parent Dataset not long enough")
        super(FullTrainningDataset, self).__init__()
        
    def __len__(self):        
        return self.length
    
    def __getitem__(self, i):
        # label = torch.from_numpy(self.y_train[index])
        return self.full_ds[i+self.offset]
    
validationRatio=0.11    

def trainTestSplit(dataset, val_share=validationRatio):
    val_offset = int(len(dataset)*(1-val_share))
    print ("Offest:" + str(val_offset))
    return FullTrainningDataset(dataset, 0, val_offset), FullTrainningDataset(dataset, 
                                                                              val_offset, len(dataset)-val_offset)

In [8]:
from torch.utils.data import TensorDataset, DataLoader

train_ds = CdiscountDataset(TRAIN_BSON_FILE, meta_data,transformations)
train_loader = DataLoader(train_ds, batch_size=BS,num_workers=N_THREADS, shuffle=True)

train_ds, val_ds = trainTestSplit(train_loader)

t_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=False,
                                            num_workers=0)
v_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0)

print (t_loader)
print (v_loader)

In [9]:
dropout = torch.nn.Dropout(p=0.50)
relu=torch.nn.LeakyReLU()
pool = nn.MaxPool2d(2, 2)

class ConvRes(nn.Module):
    def __init__(self,insize, outsize):
        super(ConvRes, self).__init__()
        drate = .3
        self.math = nn.Sequential(
                 nn.BatchNorm2d(insize),
                 nn.Dropout(drate),
                 torch.nn.Conv2d(insize, outsize, kernel_size=2,padding=2),
                 nn.PReLU(),
                )
        
    def forward(self, x):
        return self.math(x) 

class ConvCNN(nn.Module):
    def __init__(self,insize, outsize, kernel_size=7, padding=2, pool=2, avg=True):
        super(ConvCNN, self).__init__()
        self.avg=avg
        self.math = torch.nn.Sequential(
            torch.nn.Conv2d(insize, outsize, kernel_size=kernel_size,padding=padding),
            torch.nn.BatchNorm2d(outsize),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(pool,pool),
        )
        self.avgpool=torch.nn.AvgPool2d(pool,pool)
        
    def forward(self, x):
        x=self.math(x)
        if self.avg is True:
            x=self.avgpool(x)
        return x   
        
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()        
        
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        
        self.cnn1 = ConvCNN (3,128,  kernel_size=7, pool=4, avg=False)
        self.cnn2 = ConvCNN (128,128, kernel_size=5, pool=2, avg=True)        
        
        self.res1 = ConvRes (128,32)
        
        self.features = nn.Sequential( 
            self.cnn1,dropout,          
            self.cnn2,            
            self.res1,
        )        
        
        self.classifier = torch.nn.Sequential(
            nn.Linear(6272, 5270),             
        )
        self.sig=nn.Sigmoid()        
            
    def forward(self, x):
        x = self.features(x) 
#         print (x.data.shape)
        x = x.view(x.size(0), -1)      
#         print (x.data.shape)
        x = self.classifier(x)     
#         print (x.data.shape)
        x = F.log_softmax(x)
        return x        

model = Net()
# print(model)

In [ ]:
# loss_func=torch.nn.BCELoss() # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()
loss_func = criterion
### Train

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

LR = 0.0005
MOMENTUM= 0.95
num_epoches=10

# optimizer = torch.optim.Adam(model.parameters(), lr=LR,weight_decay=5e-5) #  L2 regularization

if use_cuda:
    model.cuda()
    loss_func.cuda()

lgr.info (optimizer)
lgr.info (loss_func)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

criterion = loss_func
all_losses = []
val_losses = []


if __name__ == '__main__':

    for epoch in range(num_epoches):
        print('Epoch {}'.format(epoch + 1))
        print('*' * 5 + ':')
        running_loss = 0.0
        running_acc = 0.0
        for i, data in enumerate(train_loader, 1):

            img, label = data
            if use_cuda:
                img, label = Variable(img.cuda(async=True)), Variable(label.cuda(async=True))  # On GPU
            else:
                img, label = Variable(img), Variable(
                    label)  # RuntimeError: expected CPU tensor (got CUDA tensor)

            out = model(img)
            loss = criterion(out, label)
            running_loss += loss.data[0] * label.size(0)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 10 == 0:
                all_losses.append(running_loss / (batch_size * i))
                print('[{}/{}] Loss: {:.6f}'.format(
                    epoch + 1, num_epoches, running_loss / (batch_size * i),
                    running_acc / (batch_size * i)))

        print('Finish {} epoch, Loss: {:.6f}'.format(epoch + 1, running_loss / (len(train_ds))))

    #     model.eval()
    #     eval_loss = 0
    #     eval_acc = 0
    #     for data in v_loader:
    #         img, label = data

    #         if use_cuda:
    #             img, label = Variable(img.cuda(async=True), volatile=True),
    #             Variable(label.cuda(async=True), volatile=True)  # On GPU
    #         else:
    #             img = Variable(img, volatile=True)
    #             label = Variable(label, volatile=True)

    #         out = model(img)
    #         loss = criterion(out, label)
    #         eval_loss += loss.data[0] * label.size(0)

    #     print('VALIDATION Loss: {:.6f}'.format(eval_loss / (len(val_ds))))
    #     val_losses.append(eval_loss / (len(val_ds)))
    #     print()
